In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

import time
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image, ImageDraw
import random

Using TensorFlow backend.


In [2]:
#Random start of population
#genomes as a list of gene sequences
def initialise():
    genomes =[]
    for i in range(8):
        genomes.append([np.random.randint(2, size=6)]) # add 5 randomly initialised indiviudals
    
    #entschachtelungsprozess
    ent = [5]*len(genomes)
    i = 0
    for k in genomes:
        for l in k:
            
            ent[i] =l
            i = i+1
    

    return ent

    

In [ ]:
#usefull functions

def paring(mom,dad):
    return(np.concatenate((mom[:3], dad[3:]), axis=0))

def entschachteln(genos):
    print(len(genos))
    neue = [0,0]
    i = 0
    for k in genos:
        for l in k:
            print(l)
            neue[i] =l
            i = i+1

    return (neue)

def mutation(popul):
    position = random.randrange(0, len(popul))
    gencode = popul[position]
    
    decider = random.uniform(0, 1)
    posi = 0
    if decider < 0.1 :
        print('Mutation')
        posi = random.randrange(0, len(gencode))
    
        if gencode[posi] == 1:
            gencode[posi] = 0
        elif gencode[posi] == 0:
            gencode[posi] = 1
            
    popul[position] = np.array(gencode)
    
    return popul

''' A one to one replacement reproduction '''
def selected(maxis, actualpop): 
    childs= []
    maxis= np.array(maxis)
    indices = maxis.argsort()[-4:][::-1]
    print('Highest acc at')
    print(indices)
    parents= []
    
   
    for ind in indices:    
        parents.append(actualpop[ind])
            
    print('PPPPPP')
    print(parents)
    random.shuffle(parents)
    f= 0
    print(parents)
    
    #### was passiert ungerade zahl geht nicht und 50 eltern überleben
    # 1 un2 2un3
    while f < len(parents)-1: 
        mom = parents[f]
        f= f +1
        dad = parents[f]     
        child = paring(mom,dad)
        childs.append(child)
        child = paring(dad,mom)
        childs.append(child)
        
        
    return childs

In [4]:
def netzdurchlauf(acutalpop):
    accuracy_of_population= []
    
    print('Aktuelle Population')
    print(actualpop)
    for element in actualpop:
        batch_size = 128
        num_classes = 10
        epochs = 12

        # input image dimensions
        img_rows, img_cols = 28, 28

        # the data, split between train and test sets
        (x_train, y_train), (x_test, y_test) = mnist.load_data()

        if K.image_data_format() == 'channels_first':
            x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
            x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
            input_shape = (1, img_rows, img_cols)
        else:
            x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
            x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
            input_shape = (img_rows, img_cols, 1)

        x_train = x_train.astype('float32')
        x_test = x_test.astype('float32')
        x_train /= 255
        x_test /= 255
#             print('x_train shape:', x_train.shape)
#             print(x_train.shape[0], 'train samples')
#             print(x_test.shape[0], 'test samples')

        # convert class vectors to binary class matrices
        y_train = keras.utils.to_categorical(y_train, num_classes)
        y_test = keras.utils.to_categorical(y_test, num_classes)

        model = Sequential()




        model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
            ########
            
            

        for gen in element:
            if (gen== 0):
                model.add(Conv2D(64, (3, 3), activation='relu'))
            elif (gen == 1):
                model.add(Dense(128, activation='relu'))




        model.add(Flatten())
        model.add(Dropout(0.5))
        model.add(Dense(num_classes, activation='softmax'))

        model.compile(loss=keras.losses.categorical_crossentropy,
                          optimizer=keras.optimizers.Adadelta(),
                          metrics=['accuracy'])

        model.fit(x_train, y_train,
                      batch_size=batch_size,
                      epochs=epochs,
                      verbose=1,
                      validation_data=(x_test, y_test))
        score = model.evaluate(x_test, y_test, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])
        accuracy_of_population.append(score[1])
        
    return (accuracy_of_population)

In [5]:
def fortpflanzung (maxis, actualpop):
    childs= []
    maxis= np.array(maxis)
    indices = maxis.argsort()[-3:][::-1]
    print('Highest acc at')
    print(indices)
    parents= []
    
    
    for ind in indices:    
        parents.append(actualpop[ind])
   
    
    print('PPPPPP')
    print(parents)
    random.shuffle(parents)
    f= 0
    print(parents)
    #parents = entschachteln(parents)
#     while f < len(parents)-1: 
#         mom = parents[f]
#         f= f +1
#         dad = parents[f]    
        
# #         decider = random.uniform(0, 1) # decides random how the cross-over works
# #         if decider < 0.5 :
# #             child = paring(mom,dad)
# #             childs.append(child)
# #         else:
# #             child = paring(dad,mom)
# #             childs.append(child)

    
#         child = paring(mom,dad)
#         childs.append(child)
    
#             child = paring(dad,mom)
#             childs.append(child)

    while f < len(parents)-1: 
        mom = parents[f]
        f= f +1
        dad = parents[f]     
        child = paring(mom,dad)
        childs.append(child)
        child = paring(dad,mom)
        childs.append(child)
    print('CCCCCCC')
    print(childs)
    parents = np.concatenate((parents, childs), axis=0) 
    
    print('OVERALL')
    print(parents)
    return parents

In [ ]:

# Hauptmethode
start = time.time()


DURCHGÄNGE = 1 # variable to clarify number of generations
actualpop= initialise()

#actualpop= entschachteln(actualpop)
evalaccuris = []
saved = actualpop
i = 1
    
while i < DURCHGÄNGE+1:
    
    nummer = i
    accuri = netzdurchlauf(actualpop)
    print(' Accuracy of populationnummer: ')
    print( nummer)
    print(accuri)
    print('highest accuracy of populationnummer: ')
    print( nummer)
    print(np.amax(accuri))
    #print(actualpop)
    actualpop = fortpflanzung(accuri, actualpop)
    actualpop = mutation(actualpop)
    
    evalaccuris.append([i ,np.amax(accuri)])
    i = i+1
    
    
    
print('The highest accuracies in population')
print(evalaccuris)  

print('Wartezeit: ')
end = time.time()
seconds = end - start
minutes = seconds / 60
print(minutes)

[[array([0, 0, 1, 0, 0, 0])], [array([0, 0, 0, 1, 0, 1])], [array([1, 1, 1, 1, 0, 1])], [array([1, 0, 1, 1, 0, 1])], [array([0, 1, 1, 0, 0, 1])], [array([1, 1, 1, 1, 0, 1])], [array([1, 0, 0, 1, 1, 1])], [array([1, 0, 0, 0, 1, 0])]]
[array([0, 0, 1, 0, 0, 0]), array([0, 0, 0, 1, 0, 1]), array([1, 1, 1, 1, 0, 1]), array([1, 0, 1, 1, 0, 1]), array([0, 1, 1, 0, 0, 1]), array([1, 1, 1, 1, 0, 1]), array([1, 0, 0, 1, 1, 1]), array([1, 0, 0, 0, 1, 0])]
Aktuelle Population
[array([0, 0, 1, 0, 0, 0]), array([0, 0, 0, 1, 0, 1]), array([1, 1, 1, 1, 0, 1]), array([1, 0, 1, 1, 0, 1]), array([0, 1, 1, 0, 0, 1]), array([1, 1, 1, 1, 0, 1]), array([1, 0, 0, 1, 1, 1]), array([1, 0, 0, 0, 1, 0])]


W0920 12:16:49.835772  1740 deprecation_wrapper.py:119] From C:\Users\vivia\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0920 12:16:49.853836  1740 deprecation_wrapper.py:119] From C:\Users\vivia\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0920 12:16:49.857972  1740 deprecation_wrapper.py:119] From C:\Users\vivia\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0920 12:16:49.915095  1740 deprecation_wrapper.py:119] From C:\Users\vivia\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0920 12:16:49.919315  1740 deprecation_wrapper.py:119] From C:\Users\vivia\Ana

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 39s 648us/step - loss: 0.4640 - acc: 0.8491 - val_loss: 0.0749 - val_acc: 0.9770
Epoch 2/12
60000/60000 [==============================] - 33s 544us/step - loss: 0.0809 - acc: 0.9769 - val_loss: 0.0580 - val_acc: 0.9814
Epoch 3/12
60000/60000 [==============================] - 32s 531us/step - loss: 0.0530 - acc: 0.9842 - val_loss: 0.0978 - val_acc: 0.9724
Epoch 4/12
60000/60000 [==============================] - 34s 559us/step - loss: 0.0430 - acc: 0.9878 - val_loss: 0.0231 - val_acc: 0.9929
Epoch 5/12
60000/60000 [==============================] - 33s 557us/step - loss: 0.0337 - acc: 0.9903 - val_loss: 0.0443 - val_acc: 0.9874
Epoch 6/12
60000/60000 [==============================] - 34s 559us/step - loss: 0.0292 - acc: 0.9917 - val_loss: 0.0207 - val_acc: 0.9940
Epoch 7/12
60000/60000 [==============================] - 33s 558us/step - loss: 0.0253 - acc: 0.9928 - val_loss: 0.

In [ ]:
#Visualisation
df = pd.DataFrame(data =evalaccuris)
df.rename(columns={0: 'Generation',1:'Accuracy'}, inplace=True)
df
sns.regplot(x=df['Generation'],y=df['Accuracy'])
plt.ylim(0.990, 1)


In [ ]:
print(accuri)